In [3]:
pip install opencv-python

     |████████████████████████████████| 46.3 MB 5.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import numpy as np

In [5]:
from PIL import Image
from PIL.ExifTags import TAGS

In [7]:
image_bright4 = Image.open('HDR_Samples/sunset/bright4.jpg')
image_bright3 = Image.open('HDR_Samples/sunset/bright3.jpg')
image_bright2 = Image.open('HDR_Samples/sunset/bright2.jpg')
image_bright1 = Image.open('HDR_Samples/sunset/bright1.jpg')
image_dark4 = Image.open('HDR_Samples/sunset/dark4.jpg')
image_dark3 = Image.open('HDR_Samples/sunset/dark3.jpg')
image_dark2 = Image.open('HDR_Samples/sunset/dark2.jpg')
image_dark1 = Image.open('HDR_Samples/sunset/dark1.jpg')
image_normal = Image.open('HDR_Samples/sunset/normal.jpg')
img_list = [image_bright4,image_bright3,image_bright2,image_bright1,image_normal,image_dark4,image_dark3,image_dark2,image_dark1]

In [8]:
building_bright1 = Image.open('HDR_Samples/drone/bright1.jpg')
building_bright2 = Image.open('HDR_Samples/drone/bright2.jpg')
building_dark1 = Image.open('HDR_Samples/drone/dark1.jpg')
building_dark2 = Image.open('HDR_Samples/drone/dark2.jpg')
building_normal = Image.open('HDR_Samples/drone/normal.jpg')
building_img_list = [building_bright1,building_bright2,building_normal,building_dark1,building_dark2]


Reading Shutter Speed of the Images

In [24]:
shutter_speeds = []
for img in img_list:
    exifdata = img.getexif()
    print(exifdata)
    temp = exifdata.get(33434)
    shutter_speeds.append(temp[0]/temp[1])


{296: 2, 282: 240.0, 272: 'NIKON Z 6', 305: 'Adobe Photoshop Lightroom Classic 11.0 (Macintosh)', 306: '2022:04:16 22:18:21', 34665: 226, 283: 240.0, 271: 'NIKON CORPORATION'}


TypeError: 'int' object is not subscriptable

In [14]:
building_ss = []
for img in building_img_list:
  exifdata = img.getexif()
  temp = exifdata.get(33434)
  building_ss.append(temp[0]/temp[1])

TypeError: 'NoneType' object is not subscriptable

In [ ]:
exposure_times = np.array(shutter_speeds, dtype=np.float32)

In [ ]:
building_exposure_times = np.array(building_ss, dtype=np.float32)

In [ ]:
img_bright4 = cv2.imread('HDR_Samples/sunset/bright4.jpg')
img_bright3 = cv2.imread('HDR_Samples/sunset/bright3.jpg')
img_bright2 = cv2.imread('HDR_Samples/sunset/bright2.jpg')
img_bright1 = cv2.imread('HDR_Samples/sunset/bright1.jpg')
img_dark4 = cv2.imread('HDR_Samples/sunset/dark4.jpg')
img_dark3 = cv2.imread('HDR_Samples/sunset/dark3.jpg')
img_dark2 = cv2.imread('HDR_Samples/sunset/dark2.jpg')
img_dark1 = cv2.imread('HDR_Samples/sunset/dark1.jpg')
img_normal = cv2.imread('HDR_Samples/sunset/normal.jpg')
image_list = [img_bright4,img_bright3,img_bright2,img_bright1,img_normal,img_dark4,img_dark3,img_dark2,img_dark1]

In [ ]:
building_img_bright1 = cv2.imread('HDR_Samples/bright1.jpg')
building_img_bright2 = cv2.imread('HDR_Samples/bright2.jpg')
building_img_dark1 = cv2.imread('HDR_Samples/dark1.jpg')
building_img_dark2 = cv2.imread('HDR_Samples/dark2.jpg')
building_img_normal = cv2.imread('HDR_Samples/normal.jpg')
building_image_list = [building_img_bright1,building_img_bright2,building_img_normal,building_img_dark1,building_img_dark2]

In [ ]:
alignMTB = cv2.createAlignMTB()
alignMTB.process(image_list, image_list)


In [ ]:
#alignMTB.process(building_image_list,building_image_list)

In [ ]:
merge_debevec = cv2.createMergeDebevec()
hdr_debevec = merge_debevec.process(image_list, times=exposure_times.copy())

In [ ]:
hdr_debevec_1 = merge_debevec.process(building_image_list, times=building_exposure_times.copy())

Tone Mapping

In [ ]:
tonemap1 = cv2.createTonemap(gamma=2.2)
res_debevec = tonemap1.process(hdr_debevec.copy())

In [ ]:
res_debevec_building = tonemap1.process(hdr_debevec_1.copy())

In [ ]:
res_debevec_8bit = np.clip(res_debevec*255, 0, 255).astype('uint8')
cv2.imwrite("ldr_debevec.jpg", res_debevec_8bit)

True

In [ ]:
res_debevec_8bit_building = np.clip(res_debevec_building*255, 0, 255).astype('uint8')
cv2.imwrite("ldr_debevec_building.jpg", res_debevec_8bit_building)

True

In [ ]:
tonemapDrago = cv2.createTonemapDrago(1.2, 1)
#adjusting the 2nd parameter adjusts the contrast
ldrDrago = tonemapDrago.process(hdr_debevec)
cv2.imwrite("ldr-Drago.jpg", ldrDrago * 255)


True

In [ ]:
tonemapDrago_building = cv2.createTonemapDrago(1.0, 0.5)
ldrDrago_building = tonemapDrago.process(hdr_debevec_1)
cv2.imwrite("ldr-Drago-building.jpg", ldrDrago_building * 255)


True

In [ ]:
tonemapReinhard = cv2.createTonemapReinhard(1, 2,0,1)
ldrReinhard = tonemapReinhard.process(hdr_debevec)
cv2.imwrite("ldr-Reinhard.jpg", ldrReinhard * 255)


True

In [ ]:
#auto contrast
ldr_image = cv2.imread('HDR_Samples/building/ldr.jpg')
alpha = 1.1
beta = 0
auto_contrast_image = cv2.convertScaleAbs(ldr_image, alpha=alpha, beta=beta)
cv2.imwrite("auto-contrast.jpg", auto_contrast_image)

True